In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import tgt
import os
import numpy as np
from tqdm import tqdm
import librosa
import glob

sys.path.append("../")
from pyfiles.feature_extractor import wav2vecExtractor

In [2]:
dataset_dir = "/mntcephfs/lee_dataset/tts/LibriTTS_R/"
feat_base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/LibriTTS_R/features/"

wav2vec = wav2vecExtractor()
speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*/*")]
speakers.sort()

In [23]:
save = True

files = []
for spk in tqdm(speakers):
    files += glob.glob(dataset_dir + f"*/{spk}/*/*.wav")
files.sort()
for path in tqdm(files):
    embedding = wav2vec.get_feature(path).T.detach().cpu().numpy()
    savepath = feat_base_dir + "/".join(path.split("/")[-4:])[:-4] + "_wav2vec.npy"
    if save:
        np.save(savepath, embedding)

  0%|          | 8/160267 [00:02<13:31:20,  3.29it/s]Exception ignored in: <generator object tqdm.__iter__ at 0x1554addc3190>
Traceback (most recent call last):
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/site-packages/tqdm/std.py", line 1196, in __iter__
    self.close()
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/site-packages/tqdm/std.py", line 1302, in close
    self.display(pos=0)
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/site-packages/tqdm/std.py", line 1495, in display
    self.sp(self.__str__() if msg is None else msg)
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/site-packages/tqdm/std.py", line 459, in print_status
    fp_write('\r' + s + (' ' * max(last_len[0] - len_s, 0)))
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/site-packages/tqdm/std.py", line 453, in fp_write
    fp_flush()
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk

# Normalize

In [24]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib

In [26]:
scaler_name = "LibriTTS-R_wav2vec"

In [30]:
scaler = StandardScaler()
files = []
for spk in tqdm(speakers):
    files += glob.glob(feat_base_dir + f"train*/{spk}/*/*[0-9]_wav2vec.npy")
files.sort()
for path in tqdm(files):
    mel = np.load(path)
    scaler.partial_fit(mel.T)
            
scaler_filename = f"ckpts/scalers/{scaler_name}.save"
joblib.dump(scaler, scaler_filename) 
# a = joblib.load(scaler_filename)

100%|██████████| 333/333 [00:01<00:00, 287.49it/s]
